## Imports

In [1]:
library(dplyr)
library(tidyr)
library("psych")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Monde

In [2]:
## Chargement des données

covid_csv <- read.csv("https://raw.githubusercontent.com/gibello/whocovid19/master/global_who_data.csv")
pop_csv <- read.csv("https://raw.githubusercontent.com/datasets/population/master/data/population.csv")
country_csv <- read.csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")
pop <- filter(pop_csv, Year == max(pop_csv$Year))
# pop <- pop_csv %>% slice(which(pop_csv$Year == max(pop_csv$Year)))
# head(pop)

In [3]:
## Construction de la dataframe par jointure

df <- left_join(covid_csv, pop, by = c("ISO.3166.code" = "Country.Code"))
df <- left_join(df, country_csv, by = c("ISO.3166.code" = "alpha.3"))
# head(df)
# for(column in colnames(df)){
#     print(column)
#     print(head(df[[column]]))
#     print(sum(is.na(df[[column]])))
#     print("===================")
# }

## Data pre-processing

df$intermediate.region.code <- NULL
df <- df[complete.cases(df), ]
# head(df$Date)
df$Date <- as.Date(df$Date , format = "%Y-%m-%d")
# head(df$Date)
last_day <- max(df$Date)
# last_day
# str(df)
df$Logarithm.of.confirmed.cases <- log(df$Confirmed.cases)
# head(df$Logarithm.of.confirmed.cases)
df$Logarithm.of.deaths <- log(df$Deaths)
# head(df$Logarithm.of.deaths)
df$Population <- df$Value
df$Value <- NULL
df$Proportion.of.deaths.x1M <- (df$Deaths / df$Population * 1000000)
df$Proportion.of.confirmed.cases.x1M <- df$Confirmed.cases / df$Population * 1000000
df$Logarithm.of.proportion.of.deaths.x1M <- log(df$Deaths / df$Population * 1000000)
df$Logarithm.of.proportion.of.confirmed.cases.x1M <- log(df$Confirmed.cases / df$Population * 1000000)
last_14_days <- unique(with(df, df[(Date >= last_day - 14), ])$Date)
# df$Date %>% filter(df$Date >= last_day - 14)
# df[df$Date %in% unique(as.Date(format(df$Date + 28, "%Y-%m-%d")) - 1),]
# last_14_days

# str(df)

In [4]:
str(df)

'data.frame':	18479 obs. of  27 variables:
 $ Date                                          : Date, format: "2020-03-01" "2020-03-01" ...
 $ Country                                       : chr  "Republic of Korea" "Japan" "Singapore" "Australia" ...
 $ ISO.3166.code                                 : chr  "KOR" "JPN" "SGP" "AUS" ...
 $ Confirmed.cases                               : int  4212 254 106 27 24 16 3 1 1 1689 ...
 $ New.cases                                     : int  476 15 4 2 0 0 0 0 0 561 ...
 $ Deaths                                        : int  22 6 0 1 0 0 1 0 0 35 ...
 $ New.deaths                                    : int  4 1 0 1 0 0 0 0 0 6 ...
 $ Transmission.type                             : chr  "Local transmission" "Local transmission" "Local transmission" "Local transmission" ...
 $ Days.since.last.case                          : int  0 0 0 0 3 18 27 35 3 0 ...
 $ Country.Name                                  : chr  "Korea, Rep." "Japan" "Singapore" "Australi

In [7]:
describeFast(df)

ERROR: Error in describeFast(df): impossible de trouver la fonction "describeFast"


## Graphiques

# France (si on a le temps)

In [22]:
# df_jdd = read.csv("https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv", sep=";")

In [23]:
# dfs <- c()
# names(dfs) <- c()
# urls <- as.list(as.data.frame(t(df_jdd["url"][1])))

In [24]:
# for(url in urls){
#     if(isFALSE(grepl(url, "metadonnee", fixed=TRUE))){
#         tryCatch({
#             append(dfs, read_excel(url))
#             message("Opened the following url with Excel (1) : ")
#             message(url)
#             print(head(read_excel(url)))
#             message("")
#         }, error = function(error_condition) {
#             tryCatch({
#                 append(dfs, read.csv(url, sep=";"))
#                 message("Opened the following url with CSV (;) : ")
#                 message(url)
#                 print(head(read.csv(url, sep=";")))
#                 message("")
#             }, error = function(error_condition2) {
#                 tryCatch({
#                     append(dfs, read.csv(url, sep=","))
#                     message("Opened the following url with CSV (,) : ")
#                     message(url)
#                     print(head(read.csv(url, sep=",")))
#                     message("")
#                 }, error = function(error_condition3) {
#                     message("Can't open the following URL : ")
#                     message(url)
#                 })
#             })
#         }, finally={
#             append(names(dfs), tail(strsplit(url, "/"), n=1))
#         })
#     }
# }